In [5]:
import glob
import os
import sys
sys.path.append("..")
import pandas as pd
from pandas.io.parsers import read_csv

from utils.configuration import Configuration

config = Configuration()
def add_file_name(csv, path):
    file_name = os.path.basename(path).split('.')[0]
    csv['config'] = file_name.split('result_')[1]
    return csv

In [7]:
def read_results():
    desc_map_files = []
    for path in glob.glob('../'+config.results_dir + "/*.csv"):
        csv = read_csv(path, encoding='latin-1')
        csv = add_file_name(csv, path)
        desc_map_files.append(csv)
    results= pd.concat(desc_map_files, axis=0, ignore_index=True)
    results.drop('target_app', inplace=True, axis=1)
    results.drop('src_app', inplace=True, axis=1)
    results.fillna(0, inplace=True)
    return results

In [8]:
def config_average_performance(results):
    return results.groupby('config').mean().sort_values(by = ['accuracy'], ascending=False)

In [9]:
results = read_results()
agg_results = config_average_performance(results).sort_values(by = ['f1_score'], ascending=False)

In [15]:
# config_stat = read_csv('../'+config.analyse_dir + '/configs_status.csv')
# config_stat.set_index('config', inplace=True)
# final_result = config_stat.join(agg_results)
final_result = agg_results
final_result

,tp,tn,fp,fn,effort,accuracy,precision,recall,f1_score,reduction
config,,,,,,,,,,
wm_googleplay_craftdroid_intersection,1.650000,0.833333,2.133333,1.416667,2.683333,0.463784,0.441627,0.483333,0.456723,0.355278
jaccard_empty_craftdroid_atm,1.666667,0.833333,2.266667,1.400000,2.700000,0.459641,0.439127,0.480278,0.453987,0.340833
wm_blogs_craftdroid_intersection,1.610169,0.847458,2.254237,1.457627,2.779661,0.450523,0.428773,0.470339,0.443749,0.336723
use_standard_craftdroid_craftdroid,1.583333,0.833333,2.316667,1.483333,2.816667,0.439224,0.416627,0.459444,0.432228,0.322222
use_standard_craftdroid_union,1.583333,0.833333,2.366667,1.483333,2.866667,0.438840,0.416627,0.459444,0.432228,0.312222
...,...,...,...,...,...,...,...,...,...,...
w2v_blogs_atm_union,1.237288,0.847458,2.661017,1.830508,3.220339,0.353618,0.324899,0.359322,0.336496,0.220056
fast_googleplay_atm_union,1.233333,0.833333,2.700000,1.833333,3.233333,0.352535,0.323466,0.358889,0.334825,0.213056
bert_standard_atm_union,1.250000,0.833333,2.583333,1.816667,3.150000,0.352245,0.321243,0.359444,0.334139,0.231944


In [16]:
configs_mrr = read_csv('../' + config.analyse_dir + '/mrr.csv')
configs_mrr['config'] = configs_mrr[['word_embedding', 'training_set', 'algorithm','descriptors']].agg('_'.join, axis=1)
configs_mrr.drop(columns=['descriptors','word_embedding', 'training_set', 'algorithm'], inplace=True)
configs_mrr.set_index('config', inplace=True)
final_result = final_result.join(configs_mrr).sort_values(by = ['f1_score'], ascending=False)
# configs_mrr.loc[configs_mrr['config'] == 'glove_android_atm_union']
final_result.to_csv('../' + config.analyse_dir + '/final_results.csv')